In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import h5py
from pathlib import Path
import os
import sys
sys.path.append(os.getcwd() + '/../..')
sys.path.append(os.getcwd() + '/..')
from filepaths import filepath_from_ml_artifacts_uri
from analysis.iclr.plotting import settings, fig_size, cb_line_cycler, cb_line_cycler_solid, cb_marker_cycler, savefig
from plot_hdf5_dataset import plot_dataset
from iclr_paper_path import get_paper_path
import matplotlib
settings()
import matplotlib.pyplot as plt

path_dataset = filepath_from_ml_artifacts_uri('mlflow-artifacts:/829789035554334690/2ca1dc3ab4ca4fda96dc072fea11451f/artifacts/dataset.hdf5')

In [2]:
dataset = h5py.File(path_dataset, 'r')
print(dataset.keys())

<KeysViewHDF5 ['common_test', 'common_validation', 'controls_names', 'outputs_names', 'parameters_names', 'states_der_names', 'states_names', 'test', 'time', 'train', 'validation']>


In [3]:
dataset['common_test'].keys()

<KeysViewHDF5 ['controls', 'kl_lat_controls', 'kl_lat_controls_per_dim', 'kl_lat_parameters', 'kl_lat_state_0', 'kl_lat_state_0_per_dim', 'kl_lat_states', 'kl_lat_states_per_dim', 'kl_loss', 'lat_controls_logvar', 'lat_controls_mu', 'lat_dim_controls_populated', 'lat_dim_parameters_populated', 'lat_dim_state_0_populated', 'lat_dim_states_populated', 'lat_state_0_logvar', 'lat_state_0_mu', 'lat_states_mu', 'loss', 'ode_calls_forward', 'outputs', 'outputs_hat', 'parameters', 'reconstruction_loss', 'reconstruction_loss_outputs', 'reconstruction_loss_state_0', 'reconstruction_loss_states', 'rmse_outputs', 'rmse_state_0', 'rmse_states', 'states', 'states_der', 'states_hat', 'time_odeint', 'time_outputs']>

In [4]:
dataset_type = 'common_test'
kl_lat_state_0_per_dim = dataset[dataset_type]['kl_lat_state_0_per_dim'][:]
lat_states_mu = dataset[dataset_type]['lat_states_mu'][:]
kl_lat_states = dataset[dataset_type]['kl_lat_states_per_dim'][:]
states = dataset[dataset_type]['states'][:]
states_hat = dataset[dataset_type]['states_hat'][:]
time = dataset['time'][:]

In [5]:
n_samples = kl_lat_state_0_per_dim.shape[0]
print(n_samples)

246


In [6]:
# calculate error normalized by mean
error = np.abs(states - states_hat)
mean_per_dim = np.mean(states, axis=(0,2))
# mean_per_dim = np.std(states, axis=(0,2))
error_normalized = np.divide(error, np.expand_dims(mean_per_dim, axis=(0,2)))
# error_normalized = error_normalized[0]
rmse_total_normalized = np.sqrt(np.mean(error_normalized**2))
max_error_normalized = np.max(error_normalized)
print(rmse_total_normalized)
print(max_error_normalized)

0.0043077055188780085
0.07288027666136565


In [7]:
error_normalized[0,0,0:10]

array([0.00843493, 0.01011799, 0.01024848, 0.01118202, 0.0121044 ,
       0.01270935, 0.01321897, 0.01361283, 0.01305825, 0.01283049])

In [13]:
fig = plt.figure(figsize=fig_size(0.75,n_figs_per_width=2), tight_layout=True)
fontsize = 7

grid_spec = matplotlib.gridspec.GridSpec(1, 4, width_ratios=[1,1,1,0.05], wspace=0.2, top=1.0, left=0.00, right=1.0, bottom=0.0)
ax = [fig.add_subplot(grid_spec[i]) for i in range(4)]
for i in range(1,3):
    # share y axis
    ax[i].sharey(ax[0])
    # remove y ticks
    ax[i].set_yticks([])
# fig, ax = plt.subplots(1, 2, figsize=fig_size(2.0),)
cmap = 'RdBu'

ax[0].imshow(states[0,:,:], aspect='auto', cmap = cmap, interpolation= 'none')
# add horizontal lines on y axis
for i in range(1, states.shape[1]):
    ax[0].axhline(i-0.5, color='grey', lw=0.5)
# ax[0].set_xlabel('time step $k$', fontsize=fontsize)
ax[0].set_ylabel('state dim. $\square_i$', fontsize=fontsize)
ax[0].set_title(r'$\boldsymbol{x}_{0:T}$', fontsize=fontsize, loc='center')

ax[1].imshow(lat_states_mu[0,:, :], aspect='auto', cmap = cmap, interpolation= 'none', vmin=-2, vmax=2)
# add horizontal lines on y axis
for i in range(1, lat_states_mu.shape[1]):
    ax[1].axhline(i-0.5, color='grey', lw=0.5)
# ax[1].set_xlabel(r'time step $k$', fontsize=fontsize)
ax[1].set_title(r'$\boldsymbol{x}^z_{0:T}$', fontsize=fontsize, loc='center')

ax[2].imshow(states_hat[0,:,:], aspect='auto', cmap = cmap, interpolation= 'none')
# add horizontal lines on y axis
for i in range(1, states.shape[1]):
    ax[2].axhline(i-0.5, color='grey', lw=0.5)
# ax[2].set_xlabel('time step $k$', fontsize=fontsize)
# ax[2].set_ylabel('State Dimension', fontsize=fontsize)
ax[2].text(100, -1.6, r'$\hat{\boldsymbol{x}}_{0:T}$', fontsize=fontsize, ha ='left', va='bottom')

# # add colorbar to the right
cbar = plt.colorbar(ax[1].images[0], cax=ax[-1], orientation='vertical')
cbar.set_label(r'\noindent $x^z_k$', fontsize=fontsize, labelpad =-4.0)
cbar.ax.tick_params(labelsize=8)

# replace x ticks with time
n_ticks = 3
idx = np.linspace(0, time.shape[0], n_ticks)
_time = idx * (max(time)-min(time)) / time.shape[0]
for i in range(3):
    ax[i].set_xticks(np.linspace(0, time.shape[0], n_ticks))
    ax[i].set_xticklabels(['0', '0.5', '1'])
    ax[i].set_xlabel('time [s]', fontsize=fontsize)

# add rmse and max error
str = 'RMSE' + f'={rmse_total_normalized*100:.1f}\%' + '\n' + 'max  ' + f'={max_error_normalized*100:.1f}\% \n(test set; {n_samples} samples)'
ax[2].text(260, -0.5, str, ha='left', va='bottom', fontsize=fontsize-2)

# set all x ticks to fontsize
for i in range(3):
    ax[i].tick_params(axis='x', labelsize=fontsize)

savefig(fig, get_paper_path('BNODE', 'reconstruction_BNODE'))

finding path that contains "code", then going up one level to find "LaTeX"
e:\projects\balanced_neural_odes_internal\code\python\analysis\iclr
e:\projects\balanced_neural_odes_internal\code\python\analysis
e:\projects\balanced_neural_odes_internal\code\python
e:\projects\balanced_neural_odes_internal\code
e:\projects\balanced_neural_odes_internal
E:\projects\balanced_neural_odes_internal\LaTeX\ICLR 2025 Template\figures\BNODE\reconstruction_BNODE.png


e:\projects\balanced_neural_odes_internal\code\python\analysis\iclr\plotting.py:58: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.savefig(_path.with_suffix(f".{file_type}"), **kwargs, bbox_inches='tight', pad_inches=0)


E:\projects\balanced_neural_odes_internal\LaTeX\ICLR 2025 Template\figures\BNODE\reconstruction_BNODE.svg


c:\Users\userXXXjuliu\.conda\envs\ma_userXXX\lib\site-packages\IPython\core\events.py:82: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  func(*args, **kwargs)
c:\Users\userXXXjuliu\.conda\envs\ma_userXXX\lib\site-packages\IPython\core\pylabtools.py:152: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.canvas.print_figure(bytes_io, **kw)
